In [15]:
import vgg16bn
#import vgg16bn; reload (vgg16bn);
from vgg16bn import Vgg16BN
#from keras.applications.vgg16 import VGG16

In [6]:
#import vgg16bn
import numpy as np
#import vgg16bn; reload (vgg16bn);
#from vgg16bn import Vgg16BN
from utils import *
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model

In [7]:
batch_size=64
num_classes = 120
image_size = 224
num_channels = 3

In [9]:
train_datagen = ImageDataGenerator(rotation_range=15, shear_range=0.1, channel_shift_range=20,
                                    width_shift_range=0.1,  height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory('../data/cropped/train', target_size=(224,224),
            class_mode='categorical', shuffle=True, batch_size=batch_size)

validation_generator = test_datagen.flow_from_directory('../data/cropped/test', target_size=(224,224),
            class_mode='categorical', shuffle=False, batch_size=batch_size)

Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [16]:
#base_model = VGG16(include_top=False).model
base_model = Vgg16BN(include_top=False).model
# add a global spatial average pooling layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.4)(x)
# let's add two fully-connected layer
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

/Users/cricket/Documents/ONline classes/A7 - Picture analysis/OCR-picture-recognition/Notebooks/vgg16bn.py:55: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
/Users/cricket/Documents/ONline classes/A7 - Picture analysis/OCR-picture-recognition/Notebooks/vgg16bn.py:55: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))


ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_2/MaxPool' (op: 'MaxPool') with input shapes: [?,1,112,128].

In [ ]:
model.fit_generator(train_generator, samples_per_epoch=train_generator.nb_sample, nb_epoch=10, 
                        validation_data=validation_generator, nb_val_samples=validation_generator.nb_sample)

In [ ]:
model.optimizer.lr = 1e6
model.fit_generator(train_generator, samples_per_epoch=train_generator.nb_sample, nb_epoch=10, 
                        validation_data=validation_generator, nb_val_samples=validation_generator.nb_sample)

In [ ]:
model.save_weights('vgg_model_weights.h5')